In [1]:
import requests
import urllib.parse

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns
import squarify

%matplotlib inline

In [2]:
pd.set_option( 'display.max_columns', None )
pd.set_option( 'display.max_rows', 500 )

## Loading the data

In [3]:
hog_df = pd.read_excel( './data/surveys/Anexo 2.1. Base HOGARES_FINAL_ETIQUETADA.xlsx', skiprows = 1 )

In [6]:
del hog_df[ 'VARIABLE' ]
del hog_df[ 'ENTREVISTADOR' ]
del hog_df[ 'FECHAINI' ]
del hog_df[ 'HORAINI' ]
del hog_df[ 'FECHAFIN' ]
del hog_df[ 'HORAFIN' ]
del hog_df[ 'DURACION' ]
del hog_df[ 'AUTORIZA' ]
del hog_df[ 'DANE' ]
del hog_df[ 'DAT' ]
del hog_df[ 'CNC' ]
del hog_df[ 'factor' ]

In [7]:
hog_df.shape

(4223, 96)

In [8]:
hog_df.head()

,REGISTRO,ESTRATO_GR,CIUDAD,P26,EDAD,P27,SEXO,P1,P11,P11_SI,P11_1,P11_1_SI,P11_2,P11_2_SI,P11_3,P11_3_SI,P12,P12_SI,P13,P13_SI,P14,P15,T16_1,T16_2,T16_3,T16_4,T16_5,T16_6,T16_7,T16_8,P17_1,P17_2,P17_3,P17_4,P17_5,T18_1,T18_2,T18_3,T18_4,T18_5,T18_6,T18_7,T18_8,CT18_9_COD_1,CT18_9_COD_2,T18_9_COD,T18_9,T19_1,T19_2,T19_3,T19_4,T19_5,T19_6,T19_7,T19_8,T20_1,T20_2,T20_3,T20_4,T20_5,T20_6,CT20_7_COD_1,CT20_7_COD_2,T20_7_COD,T20_7,T22_1,T22_2,T22_3,T22_4,T22_5,T22_6,P24,T25_1,T25_2,T25_3,T25_4,T25_5,T25_6,P32,P32_OTRO,P33_1,P33_2,P33_3,P33_4,P34_1,P34_2,P34_3,P34A,LOC_1,LOC_2,LOC_3,LOC_4,LOC_5,bogota,cali,medellin
0,74,3,MEDELLÍN,91,55 años o Más,Femenino,Mujer,Sí,¿Cuántas personas?,1.0,No hay menores de 5 años en el hogar,NaN,No hay mayores de 65 años en el hogar,NaN,No hay menores en edad escolar,NaN,¿Cuántas personas?,1.0,¿Cuántas personas?,1.0,Mortal,Nada probable,Seguro,N.A.,N.A.,N.A.,Inseguro,N.A.,Inseguro,N.A.,Uso de tapabocas,Cambiar de ropa al llegar a la casa,Lavado de manos con agua y jabón por más de 20...,Desinfectar los elementos que lleva del exteri...,Mantener distancia de más de un metro con otra...,Poco probable,Poco probable,No aplica,Poco probable,Poco probable,No aplica,Poco probable,Poco probable,NaN,NaN,NaN,No aplica,Fácil,N.A.,N.A.,Fácil,N.A.,Fácil,NS/NR,N.A.,No,No,No,No,No,No,NaN,NaN,NaN,No,No,No,No,No,No,NS/NR,NO,No,No,No,No,No,NS/NR,Pregrado,NaN,Oficios del hogar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comuna 16 - Belén,NaN,NaN,NaN,NaN,Zona 6
1,107,4,BARRANQUILLA,39,35 a 44 años,Femenino,Mujer,No,¿Cuántas personas?,8.0,¿Cuántas personas?,1.0,¿Cuántas personas?,4.0,¿Cuántas personas?,3.0,NaN,NaN,Ninguno,NaN,Muy grave,Poco probable,Seguro,N.A.,Inseguro,Inseguro,Inseguro,Inseguro,Inseguro,Inseguro,Lavado de manos con agua y jabón por más de 20...,Uso de tapabocas,Mantener distancia de más de un metro con otra...,Cambiar de ropa al llegar a la casa,Desinfectar los elementos que lleva del exteri...,Ya sucede,Muy probable,Muy probable,Muy probable,Muy probable,Poco probable,Poco probable,Poco probable,NaN,NaN,NaN,NS/NR,Fácil,NS/NR,Fácil,Fácil,Difícil,Fácil,Difícil,Fácil,No,No,No,No,No,No,NaN,NaN,NaN,No,No,No,No,No,No,NS/NR,NO,No,No,No,No,No,NS/NR,Técnico,TECNICO,Buscando trabajo,Oficios del hogar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Localidad Norte – Centro Histórico,NaN,NaN,NaN,NaN
2,143,2,CARTAGENA,77,55 años o Más,Masculino,Hombre,No,¿Cuántas personas?,3.0,No hay menores de 5 años en el hogar,NaN,¿Cuántas personas?,2.0,No hay menores en edad escolar,NaN,NaN,NaN,¿Cuántas personas?,2.0,Mortal,Muy probable,Inseguro,N.A.,Inseguro,Inseguro,Inseguro,Seguro,Inseguro,Inseguro,Uso de tapabocas,Lavado de manos con agua y jabón por más de 20...,NaN,NaN,NaN,Poco probable,Poco probable,No aplica,NS/NR,NS/NR,No aplica,No aplica,Poco probable,NaN,NaN,NaN,NS/NR,Fácil,NS/NR,Fácil,NS/NR,Fácil,N.A.,Fácil,NS/NR,No,No,No,No,No,No,Ninguno,NaN,NO TIENE,No,No,Sí,No,Sí,Sí,NS/NR,NO,No,No,No,No,No,NS/NR,Bachillerato incompleto,NaN,Actividades de ocio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Localidad 1 - Histórica y del Caribe Norte,NaN,NaN,NaN
3,152,4,BOGOTÁ,56,55 años o Más,Femenino,Mujer,Sí,¿Cuántas personas?,1.0,No hay menores de 5 años en el hogar,NaN,No hay mayores de 65 años en el hogar,NaN,No hay menores en edad escolar,NaN,No tuvo contacto con ninguno por más de 15 min...,NaN,Ninguno,NaN,Grave,Nada probable,Seguro,N.A.,Inseguro,Inseguro,Inseguro,N.A.,Seguro,N.A.,Uso de tapabocas,Lavado de manos con agua y jabón por más de 20...,Mantener distancia de más de un metro con otra...,Cambiar de ropa al llegar a la casa,Desinfectar los elementos que lleva del exteri...,Poco probable,Muy probable,No aplica,Poco probable,Poco probable,No aplica,Poco probable,Poco probable,NaN,NaN,NaN,No aplica,Fácil,NS/NR,N.A.,Fácil,Fácil,Difícil,Fácil,N.A.,No,No,No,No,No,No,NaN,NaN,NaN,No,NS/NR,No,No,No,No,No,NO,No,No,No,No,No,No,Bachillerato incompleto,NaN,Oficios del hogar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Engativá,NaN,NaN,NaN,NaN,Norte,NaN,NaN
4,158,2,CARTA

## Tapabocas,lavado de manos y distanciamiento

In [23]:
hog_df[ 'P17_1' ].unique()

array(['Uso de tapabocas',
       'Lavado de manos con agua y jabón por más de 20 segundos',
       'Desinfectar los elementos que lleva del exterior a la casa',
       'Cambiar de ropa al llegar a la casa', 'NS/NR',
       'Mantener distancia de más de un metro con otras personas'],
      dtype=object)

In [19]:
def tapabocas( x ):
    text = 'Uso de tapabocas'
    if x[ 'P17_1' ] == text:
        return True
    elif x[ 'P17_2' ] == text:
        return True
    elif x[ 'P17_3' ] == text:
        return True
    elif x[ 'P17_4' ] == text:
        return True
    elif x[ 'P17_5' ] == text:
        return True
    return False

hog_df[ 'tapabocas' ] = hog_df[ [ 'P17_1', 'P17_2', 'P17_3', 'P17_4', 'P17_5' ] ].apply( lambda x: tapabocas( x ), axis = 1 )

In [20]:
def lavado_manos( x ):
    text = 'Lavado de manos con agua y jabón por más de 20 segundos'
    if x[ 'P17_1' ] == text:
        return True
    elif x[ 'P17_2' ] == text:
        return True
    elif x[ 'P17_3' ] == text:
        return True
    elif x[ 'P17_4' ] == text:
        return True
    elif x[ 'P17_5' ] == text:
        return True
    return False

hog_df[ 'lavado_manos' ] = hog_df[ [ 'P17_1', 'P17_2', 'P17_3', 'P17_4', 'P17_5' ] ].apply( lambda x: lavado_manos( x ), axis = 1 )

In [22]:
def distancia_1m( x ):
    text = 'Mantener distancia de más de un metro con otras personas'
    if x[ 'P17_1' ] == text:
        return True
    elif x[ 'P17_2' ] == text:
        return True
    elif x[ 'P17_3' ] == text:
        return True
    elif x[ 'P17_4' ] == text:
        return True
    elif x[ 'P17_5' ] == text:
        return True
    return False

hog_df[ 'distancia_1m' ] = hog_df[ [ 'P17_1', 'P17_2', 'P17_3', 'P17_4', 'P17_5' ] ].apply( lambda x: distancia_1m( x ), axis = 1 )

In [32]:
hog_df[ 'CIUDAD' ].unique()

array(['MEDELLÍN', 'BARRANQUILLA', 'CARTAGENA', 'BOGOTÁ', 'CALI'],
      dtype=object)

In [33]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'MEDELLÍN', 'tapabocas' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,tapabocas
0,True,0.972569
1,False,0.027431


In [34]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BARRANQUILLA', 'tapabocas' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,tapabocas
0,True,0.975062
1,False,0.024938


In [35]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CARTAGENA', 'tapabocas' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,tapabocas
0,True,0.975
1,False,0.025


In [36]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BOGOTÁ', 'tapabocas' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,tapabocas
0,True,0.980198
1,False,0.019802


In [37]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CALI', 'tapabocas' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,tapabocas
0,True,0.965389
1,False,0.034611


In [38]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'MEDELLÍN', 'lavado_manos' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,lavado_manos
0,True,0.938903
1,False,0.061097


In [39]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BARRANQUILLA', 'lavado_manos' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,lavado_manos
0,True,0.953865
1,False,0.046135


In [40]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CARTAGENA', 'lavado_manos' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,lavado_manos
0,True,0.93625
1,False,0.06375


In [41]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BOGOTÁ', 'lavado_manos' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,lavado_manos
0,True,0.967327
1,False,0.032673


In [42]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CALI', 'lavado_manos' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,lavado_manos
0,True,0.930779
1,False,0.069221


In [47]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'MEDELLÍN', 'distancia_1m' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,distancia_1m
0,True,0.856608
1,False,0.143392


In [46]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BARRANQUILLA', 'distancia_1m' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,distancia_1m
0,True,0.885287
1,False,0.114713


In [45]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CARTAGENA', 'distancia_1m' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,distancia_1m
0,True,0.8325
1,False,0.1675


In [44]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BOGOTÁ', 'distancia_1m' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,distancia_1m
0,True,0.858416
1,False,0.141584


In [43]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CALI', 'distancia_1m' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,distancia_1m
0,True,0.820766
1,False,0.179234


## Probabilidad de contagiarse

In [52]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'MEDELLÍN', 'P15' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,P15
0,Poco probable,0.557357
1,Muy probable,0.307980
2,Nada probable,0.103491
3,NS/NR <strong>(E: NO LEER)</strong>,0.031172


In [51]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BARRANQUILLA', 'P15' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,P15
0,Poco probable,0.516209
1,Nada probable,0.245636
2,Muy probable,0.216958
3,NS/NR <strong>(E: NO LEER)</strong>,0.021197


In [50]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CARTAGENA', 'P15' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,P15
0,Poco probable,0.52000
1,Nada probable,0.25375
2,Muy probable,0.20625
3,NS/NR <strong>(E: NO LEER)</strong>,0.02000


In [49]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BOGOTÁ', 'P15' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,P15
0,Poco probable,0.512871
1,Muy probable,0.317822
2,Nada probable,0.140594
3,NS/NR <strong>(E: NO LEER)</strong>,0.028713


In [48]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CALI', 'P15' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,P15
0,Poco probable,0.514215
1,Muy probable,0.257108
2,Nada probable,0.190358
3,NS/NR <strong>(E: NO LEER)</strong>,0.038319


## Mercado

In [54]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'MEDELLÍN', 'T18_2' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,T18_2
0,Poco probable,0.493766
1,Muy probable,0.314214
2,Ya sucede,0.144638
3,No aplica,0.032419
4,NS/NR,0.014963


In [55]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BARRANQUILLA', 'T18_2' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,T18_2
0,Poco probable,0.425187
1,Muy probable,0.356608
2,Ya sucede,0.180798
3,No aplica,0.023691
4,NS/NR,0.013716


In [56]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CARTAGENA', 'T18_2' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,T18_2
0,Poco probable,0.37625
1,Muy probable,0.37375
2,Ya sucede,0.22500
3,No aplica,0.01375
4,NS/NR,0.01125


In [57]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BOGOTÁ', 'T18_2' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,T18_2
0,Poco probable,0.530693
1,Muy probable,0.321782
2,Ya sucede,0.114851
3,No aplica,0.019802
4,NS/NR,0.012871


In [58]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CALI', 'T18_2' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,T18_2
0,Poco probable,0.447466
1,Muy probable,0.352287
2,Ya sucede,0.156984
3,No aplica,0.025958
4,NS/NR,0.017305


## Mantener el trabajo

In [59]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'MEDELLÍN', 'T19_2' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,T19_2
0,Difícil,0.302993
1,Fácil,0.271820
2,Muy difícil,0.253117
3,NS/NR,0.076060
4,N.A.,0.071072
5,Muy fácil,0.024938


In [60]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BARRANQUILLA', 'T19_2' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,T19_2
0,Difícil,0.425187
1,Muy difícil,0.221945
2,Fácil,0.183292
3,N.A.,0.077307
4,NS/NR,0.067332
5,Muy fácil,0.024938


In [61]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CARTAGENA', 'T19_2' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,T19_2
0,Difícil,0.40125
1,Muy difícil,0.29875
2,Fácil,0.13500
3,N.A.,0.07875
4,NS/NR,0.07250
5,Muy fácil,0.01375


In [62]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BOGOTÁ', 'T19_2' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,T19_2
0,Difícil,0.323762
1,Fácil,0.267327
2,Muy difícil,0.212871
3,N.A.,0.087129
4,NS/NR,0.085149
5,Muy fácil,0.023762


In [63]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CALI', 'T19_2' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,T19_2
0,Difícil,0.374536
1,Muy difícil,0.233622
2,Fácil,0.201483
3,N.A.,0.101360
4,NS/NR,0.071693
5,Muy fácil,0.017305


## Teletrabajo

In [64]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'MEDELLÍN', 'P34A' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,P34A
0,NaN,0.744389
1,Teletrabajo – trabajo en casa,0.178304
2,Ir al lugar de trabajo,0.072319
3,"No trabaja, pero recibe salario-honorarios",0.004988


In [65]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BARRANQUILLA', 'P34A' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,P34A
0,NaN,0.773067
1,Teletrabajo – trabajo en casa,0.132170
2,Ir al lugar de trabajo,0.089776
3,"No trabaja, pero recibe salario-honorarios",0.004988


In [66]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CARTAGENA', 'P34A' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,P34A
0,NaN,0.81875
1,Ir al lugar de trabajo,0.09375
2,Teletrabajo – trabajo en casa,0.08375
3,"No trabaja, pero recibe salario-honorarios",0.00375


In [67]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'BOGOTÁ', 'P34A' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,P34A
0,NaN,0.669307
1,Teletrabajo – trabajo en casa,0.241584
2,Ir al lugar de trabajo,0.081188
3,"No trabaja, pero recibe salario-honorarios",0.006931
4,9,0.000990


In [68]:
hog_df.loc[ hog_df[ 'CIUDAD' ] == 'CALI', 'P34A' ].value_counts( dropna = False, normalize = True ).reset_index()

,index,P34A
0,NaN,0.778739
1,Teletrabajo – trabajo en casa,0.123609
2,Ir al lugar de trabajo,0.087763
3,"No trabaja, pero recibe salario-honorarios",0.009889


In [24]:
hog_df.head()

,REGISTRO,ESTRATO_GR,CIUDAD,P26,EDAD,P27,SEXO,P1,P11,P11_SI,P11_1,P11_1_SI,P11_2,P11_2_SI,P11_3,P11_3_SI,P12,P12_SI,P13,P13_SI,P14,P15,T16_1,T16_2,T16_3,T16_4,T16_5,T16_6,T16_7,T16_8,P17_1,P17_2,P17_3,P17_4,P17_5,T18_1,T18_2,T18_3,T18_4,T18_5,T18_6,T18_7,T18_8,CT18_9_COD_1,CT18_9_COD_2,T18_9_COD,T18_9,T19_1,T19_2,T19_3,T19_4,T19_5,T19_6,T19_7,T19_8,T20_1,T20_2,T20_3,T20_4,T20_5,T20_6,CT20_7_COD_1,CT20_7_COD_2,T20_7_COD,T20_7,T22_1,T22_2,T22_3,T22_4,T22_5,T22_6,P24,T25_1,T25_2,T25_3,T25_4,T25_5,T25_6,P32,P32_OTRO,P33_1,P33_2,P33_3,P33_4,P34_1,P34_2,P34_3,P34A,LOC_1,LOC_2,LOC_3,LOC_4,LOC_5,bogota,cali,medellin,tapabocas,lavado_manos,distancia_1m
0,74,3,MEDELLÍN,91,55 años o Más,Femenino,Mujer,Sí,¿Cuántas personas?,1.0,No hay menores de 5 años en el hogar,NaN,No hay mayores de 65 años en el hogar,NaN,No hay menores en edad escolar,NaN,¿Cuántas personas?,1.0,¿Cuántas personas?,1.0,Mortal,Nada probable,Seguro,N.A.,N.A.,N.A.,Inseguro,N.A.,Inseguro,N.A.,Uso de tapabocas,Cambiar de ropa al llegar a la casa,Lavado de manos con agua y jabón por más de 20...,Desinfectar los elementos que lleva del exteri...,Mantener distancia de más de un metro con otra...,Poco probable,Poco probable,No aplica,Poco probable,Poco probable,No aplica,Poco probable,Poco probable,NaN,NaN,NaN,No aplica,Fácil,N.A.,N.A.,Fácil,N.A.,Fácil,NS/NR,N.A.,No,No,No,No,No,No,NaN,NaN,NaN,No,No,No,No,No,No,NS/NR,NO,No,No,No,No,No,NS/NR,Pregrado,NaN,Oficios del hogar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comuna 16 - Belén,NaN,NaN,NaN,NaN,Zona 6,True,True,True
1,107,4,BARRANQUILLA,39,35 a 44 años,Femenino,Mujer,No,¿Cuántas personas?,8.0,¿Cuántas personas?,1.0,¿Cuántas personas?,4.0,¿Cuántas personas?,3.0,NaN,NaN,Ninguno,NaN,Muy grave,Poco probable,Seguro,N.A.,Inseguro,Inseguro,Inseguro,Inseguro,Inseguro,Inseguro,Lavado de manos con agua y jabón por más de 20...,Uso de tapabocas,Mantener distancia de más de un metro con otra...,Cambiar de ropa al llegar a la casa,Desinfectar los elementos que lleva del exteri...,Ya sucede,Muy probable,Muy probable,Muy probable,Muy probable,Poco probable,Poco probable,Poco probable,NaN,NaN,NaN,NS/NR,Fácil,NS/NR,Fácil,Fácil,Difícil,Fácil,Difícil,Fácil,No,No,No,No,No,No,NaN,NaN,NaN,No,No,No,No,No,No,NS/NR,NO,No,No,No,No,No,NS/NR,Técnico,TECNICO,Buscando trabajo,Oficios del hogar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Localidad Norte – Centro Histórico,NaN,NaN,NaN,NaN,True,True,True
2,143,2,CARTAGENA,77,55 años o Más,Masculino,Hombre,No,¿Cuántas personas?,3.0,No hay menores de 5 años en el hogar,NaN,¿Cuántas personas?,2.0,No hay menores en edad escolar,NaN,NaN,NaN,¿Cuántas personas?,2.0,Mortal,Muy probable,Inseguro,N.A.,Inseguro,Inseguro,Inseguro,Seguro,Inseguro,Inseguro,Uso de tapabocas,Lavado de manos con agua y jabón por más de 20...,NaN,NaN,NaN,Poco probable,Poco probable,No aplica,NS/NR,NS/NR,No aplica,No aplica,Poco probable,NaN,NaN,NaN,NS/NR,Fácil,NS/NR,Fácil,NS/NR,Fácil,N.A.,Fácil,NS/NR,No,No,No,No,No,No,Ninguno,NaN,NO TIENE,No,No,Sí,No,Sí,Sí,NS/NR,NO,No,No,No,No,No,NS/NR,Bachillerato incompleto,NaN,Actividades de ocio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Localidad 1 - Histórica y del Caribe Norte,NaN,NaN,NaN,True,True,False
3,152,4,BOGOTÁ,56,55 años o Más,Femenino,Mujer,Sí,¿Cuántas personas?,1.0,No hay menores de 5 años en el hogar,NaN,No hay mayores de 65 años en el hogar,NaN,No hay menores en edad escolar,NaN,No tuvo contacto con ninguno por más de 15 min...,NaN,Ninguno,NaN,Grave,Nada probable,Seguro,N.A.,Inseguro,Inseguro,Inseguro,N.A.,Seguro,N.A.,Uso de tapabocas,Lavado de manos con agua y jabón por más de 20...,Mantener distancia de más de un metro con otra...,Cambiar de ropa al llegar a la casa,Desinfectar los elementos que lleva del exteri...,Poco probable,Muy probable,No aplica,Poco probable,Poco probable,No aplica,Poco probable,Poco probable,NaN,NaN,NaN,No aplica,Fácil,NS/NR,N.A.,Fácil,Fácil,Difícil,Fácil,N.A.,No,No,No,No,No,No,NaN,NaN,NaN,No,NS/NR,No,No,No,No,No,NO,No,No,No,No,No,No,Bachillerato incompleto,NaN,Oficios del hoga